In [99]:
!python -m pip install -U pip
!nltk.download(vader_lexicon)

/bin/bash: -c: line 0: syntax error near unexpected token `vader_lexicon'
/bin/bash: -c: line 0: `nltk.download(vader_lexicon)'


In [108]:
!pip install pandas
!pip install numpy
!pip install python-binance
!pip install python-dotenv
!pip install bta-lib

     |████████████████████████████████| 92 kB 34 kB/s s eta 0:00:01


In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
import pickle
from datetime import datetime


import pandas as pd
import numpy as np
from datetime import datetime
from binance.client import Client
from binance.websockets import BinanceSocketManager
from twisted.internet import reactor
from datetime import datetime
import btalib

pd.set_option('display.max_rows', 500)

In [3]:
import logging

# Gets or creates a logger
logger = logging.getLogger(__name__)  

# set log level
logger.setLevel(logging.INFO)

# define file handler and set formatter
file_handler = logging.FileHandler('logfile.log')
formatter    = logging.Formatter('%(asctime)s : %(levelname)s : %(name)s : %(message)s')
file_handler.setFormatter(formatter)

# add file handler to logger
logger.addHandler(file_handler)

In [4]:
# init
api_key = os.environ.get('BINANCE_API')
api_secret = os.environ.get('BINANCE_SECRET')

client = Client(api_key, api_secret)

print(client.get_account())
print(client.get_asset_balance(asset='BTC'))
print(client.get_all_tickers())

{'makerCommission': 10, 'takerCommission': 10, 'buyerCommission': 0, 'sellerCommission': 0, 'canTrade': True, 'canWithdraw': True, 'canDeposit': True, 'updateTime': 1618173055044, 'accountType': 'SPOT', 'balances': [], 'permissions': ['SPOT']}
None
[{'symbol': 'ETHBTC', 'price': '0.04610000'}, {'symbol': 'LTCBTC', 'price': '0.00467300'}, {'symbol': 'BNBBTC', 'price': '0.01022200'}, {'symbol': 'NEOBTC', 'price': '0.00164200'}, {'symbol': 'QTUMETH', 'price': '0.00533200'}, {'symbol': 'EOSETH', 'price': '0.00230100'}, {'symbol': 'SNTETH', 'price': '0.00005657'}, {'symbol': 'BNTETH', 'price': '0.00266700'}, {'symbol': 'BCCBTC', 'price': '0.07908100'}, {'symbol': 'GASBTC', 'price': '0.00022620'}, {'symbol': 'BNBETH', 'price': '0.22179300'}, {'symbol': 'BTCUSDT', 'price': '50640.86000000'}, {'symbol': 'ETHUSDT', 'price': '2333.70000000'}, {'symbol': 'HSRBTC', 'price': '0.00041400'}, {'symbol': 'OAXETH', 'price': '0.00017780'}, {'symbol': 'DNTETH', 'price': '0.00002801'}, {'symbol': 'MCOETH',

In [5]:
coin_list = [ \
"ETH" + "USDT" ,
"BNB"+ "USDT",
"XRP"+ "USDT",
"USDT"+ "USDT",
"DOGE"+ "USDT",
"ADA"+ "USDT",
"DOT"+ "USDT",
"LTC"+ "USDT",
"BCH"+ "USDT",
"UNI"+ "USDT",
"LINK"+ "USDT",
"VET"+ "USDT",
"XLM"+ "USDT",
"USDC"+ "USDT",
"THETA"+ "USDT",
"FIL"+ "USDT",
"TRX"+ "USDT",
"WBTC"+ "USDT",
"SOL"+ "USDT",
"EOS"+ "USDT",
"KLAY"+ "USDT",
"NEO"+ "USDT",
"BSV"+ "USDT",
"XMR"+ "USDT"
]

In [5]:
for coin in coin_list:    
    print(coin + ":")
    #[x for x in client.get_all_tickers() if (x.get('symbol')+"USDT").startswith("XRP")]
    print([x for x in client.get_all_tickers() if x.get('symbol') == coin ])
    print("*****")

NameError: name 'coin_list' is not defined

In [109]:
def f_initial_load(coinlist):

    dict_all = {}
    for coin in coin_list:  
        dict_coin = {}    
        logger.info('Initial load {x}'.format(x=coin))
        try:
            klines = client.get_historical_klines(coin, Client.KLINE_INTERVAL_1HOUR, "1 Dec, 2017", "21 Apr, 2021")
        except:
            logger.error('Initial load {x} FAILED'.format(x=coin))
            continue
        for x in klines:
            ts = int(x[0]) / 1000
            dt = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
            dict_coin.update({dt:[x[0], x[4], x[5], x[8]]}) #date, price, volume, numoftrades    
        dict_all.update({coin:dict_coin})
        
    pickle_out = open("dict_all.pkl","wb")
    pickle.dump(dict_all, pickle_out)
    pickle_out.close()

    return dict_all


In [110]:
dict_all_coins = f_initial_load(coin_list)

In [7]:
# fetch 1 minute klines for the last day up until now
def f_update_coins(coinlist, dict_all=None):
    
    if dict_all is None:
        file = open("dict_all.pkl","rb")
        dict_all = pickle.load(file)

    for coin in list(dict_all.keys()):  
        dict_coin = {}
        logger.info('try to update coin {x}'.format(x=coin))
        
        try:
            klines = client.get_historical_klines(coin, Client.KLINE_INTERVAL_1HOUR, "4 day ago UTC")
        except:
            logger.error('UPDATE list of coin: {x} FAILED, seems no info'.format(x=coin))
            ##Make warning to say that such coin data is not up-to-date!!
            continue        
        dict_coin = dict_all.get(coin)
        for x in klines:
            ts = int(x[0]) / 1000
            dt = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
            dict_coin.update({dt:[x[0], x[4], x[5], x[8]]}) #date, price, volume, numoftrades    
        dict_all.update({coin:dict_coin})
    pickle_out = open("dict_all.pkl","wb")
    pickle.dump(dict_all, pickle_out)
    pickle_out.close()

    return dict_all


In [8]:
#dict_all_coins = f_update_coins(coin_list, dict_all_coins)
dict_all_coins = f_update_coins(coin_list, dict_all = None)

In [9]:
dict_all_coins.get("LINKUSDT")

{'2019-01-16 10:00:00': [1547632800000, '0.50660000', '63365.33000000', 151],
 '2019-01-16 11:00:00': [1547636400000, '0.49800000', '153229.70000000', 296],
 '2019-01-16 12:00:00': [1547640000000, '0.48870000', '185102.78000000', 315],
 '2019-01-16 13:00:00': [1547643600000, '0.48690000', '67053.33000000', 198],
 '2019-01-16 14:00:00': [1547647200000, '0.47460000', '168009.30000000', 360],
 '2019-01-16 15:00:00': [1547650800000, '0.47900000', '83996.24000000', 251],
 '2019-01-16 16:00:00': [1547654400000, '0.48000000', '78048.67000000', 228],
 '2019-01-16 17:00:00': [1547658000000, '0.50430000', '132251.61000000', 443],
 '2019-01-16 18:00:00': [1547661600000, '0.50160000', '58508.99000000', 199],
 '2019-01-16 19:00:00': [1547665200000, '0.50410000', '67946.15000000', 147],
 '2019-01-16 20:00:00': [1547668800000, '0.51270000', '102787.08000000', 275],
 '2019-01-16 21:00:00': [1547672400000, '0.51170000', '25010.78000000', 108],
 '2019-01-16 22:00:00': [1547676000000, '0.48580000', '7399

In [10]:
df_all_coins = pd.DataFrame(columns= ["coin", "dt", "price", "vol", "tradefreq"])
for coin in dict_all_coins.keys():
    dict_coin = dict_all_coins.get(coin)
    listx = []
    for dt in dict_coin.keys():
        listx.append([coin, dt, dict_coin.get(dt)[1], dict_coin.get(dt)[2], dict_coin.get(dt)[3]])
    df_all_coins = df_all_coins.append(pd.DataFrame(data = listx,
                                     columns= ["coin", "dt", "price", "vol", "tradefreq"]
                                    )
                       )
df_all_coins.dt = df_all_coins.dt.apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')  )

df_all_coins['last_time'] = 0
for coin in df_all_coins.coin.unique():
    max_dt = np.max(df_all_coins.loc[df_all_coins.coin == coin, 'dt'])
    df_all_coins.loc[(df_all_coins.coin == coin) & (df_all_coins.dt == max_dt ) , 'last_time'] = 1

df_all_coins.set_index("dt", drop=True, inplace = True )
df_all_coins.sort_index(inplace=True)
df_all_coins["is_day_end"] = 0
df_all_coins.loc[[True if x.strftime("%H:%M:%S") == '00:00:00' else False for x in list(df_all_coins.index)], "is_day_end"] = 1  
df_all_coins.price = df_all_coins.price.astype(float)
df_all_coins.tradefreq= df_all_coins.tradefreq.astype(float)
df_all_coins.vol= df_all_coins.vol.astype(float)
df_all_coins.tail()




,coin,price,vol,tradefreq,last_time,is_day_end
dt,,,,,,
2021-04-23 20:00:00,ADAUSDT,1.133710,2.861896e+07,69866.0,1,0
2021-04-23 20:00:00,DOGEUSDT,0.237395,4.447454e+08,102357.0,1,0
2021-04-23 20:00:00,XRPUSDT,1.111740,6.010545e+07,55430.0,1,0
2021-04-23 20:00:00,LINKUSDT,34.100000,2.823390e+05,16692.0,1,0
2021-04-23 20:00:00,XMRUSDT,337.260000,5.294275e+03,5087.0,1,0


In [21]:
df_all_coins

,coin,price,vol,tradefreq,last_time,is_day_end
dt,,,,,,
2017-12-01 00:00:00,ETHUSDT,421.420000,1.325873e+03,956.0,0,1
2017-12-01 00:00:00,BNBUSDT,1.911300,1.629423e+04,105.0,0,1
2017-12-01 00:00:00,NEOUSDT,31.879000,3.958592e+03,203.0,0,1
2017-12-01 01:00:00,ETHUSDT,421.720000,1.532115e+03,1037.0,0,0
2017-12-01 01:00:00,BNBUSDT,1.950000,1.354769e+04,103.0,0,0
...,...,...,...,...,...,...
2021-04-23 20:00:00,ADAUSDT,1.152120,2.284307e+07,56232.0,1,0
2021-04-23 20:00:00,DOGEUSDT,0.240877,3.629660e+08,83607.0,1,0
2021-04-23 20:00:00,XRPUSDT,1.121690,4.710344e+07,44825.0,1,0


In [11]:
df_all_coins["next1price"] = df_all_coins.groupby(['coin'])['price'].shift(-6)
df_all_coins["next2price"] = df_all_coins.groupby(['coin'])['price'].shift(-12)

df_all_coins["Target"] = 0

def set_target(row, margin):
    meanx = np.nanmean([row["next1price"], row["next2price"]])
    if meanx > row["price"] * (1.00 + margin):
        return 1
    elif meanx < row["price"] * (1.00 - margin):
        return -1
    else:
        return 0
    

df_all_coins.loc[df_all_coins.last_time == 0, "Target"] = \
        df_all_coins.loc[df_all_coins.last_time == 0, :].apply(lambda row: set_target(row, 0.05), axis = 1)


    

/root/projects/binance_first/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: Mean of empty slice
  import sys


In [12]:
for coin in df_all_coins.coin.unique():
    index_t = (df_all_coins.coin == coin)
    df_all_coins.loc[index_t, 'ma48'] = btalib.sma(df_all_coins.loc[index_t, 'price'], period=48).df.values.tolist()
    df_all_coins.loc[index_t, 'ma96'] = btalib.sma(df_all_coins.loc[index_t, 'price'], period=96).df.values.tolist()
    df_all_coins.loc[index_t, 'ma124'] = btalib.sma(df_all_coins.loc[index_t, 'price'], period=96).df.values.tolist()
    df_all_coins.loc[index_t, 'ma148'] = btalib.sma(df_all_coins.loc[index_t, 'price'], period=96).df.values.tolist()
    df_all_coins.loc[index_t, 'rsi14'] = btalib.rsi(df_all_coins.loc[index_t, 'price'], period=14).df.values.tolist()
    df_all_coins.loc[index_t, 'rsi28'] = btalib.rsi(df_all_coins.loc[index_t, 'price'], period=28).df.values.tolist()
    df_all_coins.loc[index_t, 'rsi42'] = btalib.rsi(df_all_coins.loc[index_t, 'price'], period=42).df.values.tolist()
    df_all_coins.loc[index_t, ["macd12", "signal12", "hist12"] ] = btalib.macd(df_all_coins.loc[index_t, 'price'], pfast=24, pslow=48, psignal=24).df.values.tolist()




df_all_coins.loc[df_all_coins.coin == 'DOGEUSDT', :].Target.value_counts()



 0    14558
 1      724
-1      486
Name: Target, dtype: int64

In [13]:
df_all_coins.loc[df_all_coins.coin == 'DOGEUSDT', :].tail(100)

,coin,price,vol,tradefreq,last_time,is_day_end,next1price,next2price,Target,ma48,ma96,ma124,ma148,rsi14,rsi28,rsi42,macd12,signal12,hist12
dt,,,,,,,,,,,,,,,,,,,
2021-04-19 15:00:00,DOGEUSDT,0.371500,1.089013e+09,371386.0,0,0,0.395345,0.407690,1,0.319473,0.300777,0.300777,0.300777,56.339429,57.050366,57.238189,0.023234,0.017746,0.005488
2021-04-19 16:00:00,DOGEUSDT,0.384289,1.227600e+09,445234.0,0,0,0.419130,0.403511,1,0.321902,0.303357,0.303357,0.303357,59.368644,58.622503,58.326616,0.023502,0.018206,0.005296
2021-04-19 17:00:00,DOGEUSDT,0.390157,5.824559e+08,223897.0,0,0,0.408903,0.413556,1,0.324507,0.305979,0.305979,0.305979,60.715505,59.330884,58.819312,0.023893,0.018661,0.005232
2021-04-19 18:00:00,DOGEUSDT,0.399137,6.003557e+08,243892.0,0,0,0.368786,0.411760,0,0.327063,0.308568,0.308568,0.308568,62.750172,60.406466,59.568563,0.024511,0.019129,0.005382
2021-04-19 19:00:00,DOGEUSDT,0.395509,4.524767e+08,169510.0,0,0,0.384921,0.384432,0,0.328713,0.310935,0.310935,0.310935,61.367111,59.744373,59.123310,0.024834,0.019585,0.005249
2021-04-19 20:00:00,DOGEUSDT,0.383448,5.449588e+08,195302.0,0,0,0.414458,0.388661,0,0.330416,0.313013,0.313013,0.313013,56.879117,57.569130,57.655783,0.024566,0.019984,0.004582
2021-04-19 21:00:00,DOGEUSDT,0.395345,3.991254e+08,152787.0,0,0,0.407690,0.396590,0,0.332482,0.315245,0.315245,0.315245,59.987707,59.092741,58.691872,0.024715,0.020362,0.004353
2021-04-19 22:00:00,DOGEUSDT,0.419130,9.452150e+08,356185.0,0,0,0.403511,0.386145,-1,0.334985,0.317772,0.317772,0.317772,65.363552,61.927103,60.663034,0.025698,0.020789,0.004909
2021-04-19 23:00:00,DOGEUSDT,0.408903,8.040810e+08,275762.0,0,0,0.413556,0.385259,0,0.337623,0.320132,0.320132,0.320132,61.535263,60.071188,59.414255,0.026081,0.021213,0.004868


In [150]:
index_t = (df_all_coins.coin == 'DOGEUSDT')
df_all_coins.loc[index_t, :]
btalib.sma(df_all_coins.loc[index_t, 'price'], period=48).df.values.tolist()    
#btalib.sma(df_all_coins.loc[index_t, 'price'], period=48).df  


[[nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [nan],
 [0.0036205083333333333],
 [0.003611364583333333],
 [0.0036038479166666665],
 [0.0035965687499999997],
 [0.00359078125],
 [0.0035864833333333333],
 [0.003586008333333333],
 [0.0035836541666666665],
 [0.0035806916666666663],
 [0.0035774104166666665],
 [0.003574385416666667],
 [0.0035699208333333336],
 [0.003562995833333334],
 [0.0035569958333333336],
 [0.0035526875000000003],
 [0.0035481604166666667],
 [0.0035422770833333336],
 [0.0035369562500000007],
 [0.003532212500000001],
 [0.0035285625000000004],
 [0.003524583333333334],
 [0.003522083333333334],
 [0.003519427083333334],
 [0.003517227083333334],
 [0.0035154333333

In [141]:
df_all_coins.loc[df_all_coins.coin == 'DOGEUSDT', :].head(100)

,coin,price,vol,tradefreq,last_time,is_day_end,next1price,next2price,Target,ma48
dt,,,,,,,,,,
2019-07-05 12:00:00,DOGEUSDT,0.003925,949430724.0,6487.0,0,0,0.003640,0.003819,0,NaN
2019-07-05 13:00:00,DOGEUSDT,0.003835,159004267.0,1529.0,0,0,0.003688,0.003747,0,NaN
2019-07-05 14:00:00,DOGEUSDT,0.003841,105517060.0,1241.0,0,0,0.003711,0.003740,0,NaN
2019-07-05 15:00:00,DOGEUSDT,0.003738,100804728.0,1213.0,0,0,0.003710,0.003767,0,NaN
2019-07-05 16:00:00,DOGEUSDT,0.003666,120261079.0,1416.0,0,0,0.003743,0.003745,0,NaN
2019-07-05 17:00:00,DOGEUSDT,0.003628,148274922.0,1624.0,0,0,0.003870,0.003707,0,NaN
2019-07-05 18:00:00,DOGEUSDT,0.003640,58947455.0,820.0,0,0,0.003819,0.003641,0,NaN
2019-07-05 19:00:00,DOGEUSDT,0.003688,73100952.0,974.0,0,0,0.003747,0.003660,0,NaN
2019-07-05 20:00:00,DOGEUSDT,0.003711,42377075.0,770.0,0,0,0.003740,0.003619,0,NaN


In [7]:
# valid intervals - 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M

# get timestamp of earliest date data is available
timestamp = client._get_earliest_valid_timestamp('DOGEUSDT', '1d')

# request historical candle (or klines) data
klines = client.get_historical_klines("DOGEUSDT", Client.KLINE_INTERVAL_1MINUTE, "1 day ago UTC")

In [ ]:
timestamp = client._get_earliest_valid_timestamp('BTCUSDT', '1d')
timestamp

In [10]:
klines[-1]




[1618685700000,
 '0.27480000',
 '0.27489960',
 '0.27414070',
 '0.27449730',
 '832363.00000000',
 1618685759999,
 '228516.22384700',
 318,
 '363595.00000000',
 '99811.79834750',
 '0']

In [21]:
from datetime import datetime
ts = int("1618531200000") / 1000

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

2021-04-16 00:00:00
